# Modeling
Esse Notebook visa apresentar a etapa de definição do modelo de machine learning usado para fazer a predição da ```ICU```.<br>
Após a etapa de *Feature Engineering* os dados estão prontos para serem usados de maneira a auxiliar a classificação.<br>

<a name="sumario"></a>
# Sumário
<!--ts-->
   * [Importe das Bibliotecas](#import)
   * [Leitura dos dados](#leitura)
   * [Metrica de avaliação](#metrics)
   * [Dummy Classifier](#dummy)
   * [Random Forest Classifier](#rfcsf)
   * [Features com alta correlação](#corr)
   
<!--te-->

<a name="import"></a>
# Importe das Bibliotecas

## Bibliotecas Externas

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_validate
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import RepeatedStratifiedKFold

## Bibliotecas Internas

In [7]:
import sys
sys.path.insert(1, "../src/")
from modeling import run_model_cv, many_Lazy_Classifiers

-----------------
Retornar ao [Sumário](#sumario)

<a name="leitura"></a>
# Leitura dos dados
Faz a leitura dos dados que serão usados para a modelagem.<br>
**```df : pd.DataFrame```** é o DataFrame contendo os dados que já passaram pela etapa de *feature engineering*.

In [8]:
# Leitura do raw data desse  projeto no Github
df = pd.read_csv('../data/processed/df_featured.csv', index_col='Unnamed: 0')
df.index.names = ['PATIENT_VISIT_IDENTIFIER']

# Imprime na tela todas as colunas em vez de as 10 primeiras e 10 ultimas
pd.set_option('max_columns', df.shape[1])
df.head()

,AGE_ABOVE65,AGE_SCALED,DISEASE GROUPING 1,DISEASE GROUPING 3,DISEASE GROUPING 5,HTN,IMMUNOCOMPROMISED,ALBUMIN,BE_VENOUS,FFA,GGT,GLUCOSE,HEMOGLOBIN,LACTATE,LINFOCITOS,NEUTROPHILES,P02_VENOUS,PCR,SAT02_VENOUS,SODIUM,DIMER,BLOODPRESSURE_DIASTOLIC_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,BLOODPRESSURE_SISTOLIC_MEDIAN,OXYGEN_SATURATION_MEDIAN,HEART_RATE_MIN,RESPIRATORY_RATE_MIN,BLOODPRESSURE_DIASTOLIC_MAX,BLOODPRESSURE_MAX_MEDIAN_RESIDUAL,HEART_RATE_MAX,OXYGEN_SATURATION_MAX,BLOODPRESSURE_SISTOLIC_DIFF,ICU
PATIENT_VISIT_IDENTIFIER,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0.11,0.00,0.00,0.00,1.00,0.00,0.61,-1.00,-0.74,-0.96,-0.85,-0.02,1.00,-0.66,-0.79,-0.70,-1.00,0.35,-0.14,-0.97,0.19,-0.39,-0.29,-0.28,0.84,-0.44,-0.29,-0.18,-0.02,-0.67,0.84,-1.00,0
1,0,0.00,0.00,0.00,0.00,0.00,0.00,0.61,-1.00,-0.74,-0.99,-0.89,-0.17,1.00,-0.58,-0.89,-0.70,-0.95,0.35,-0.03,-0.98,0.19,-0.46,-0.11,-0.52,1.00,-0.32,-0.36,-0.18,-0.02,-0.57,1.00,-1.00,0
2,0,0.22,0.00,0.00,0.00,0.00,0.00,0.61,-0.86,-0.74,-0.96,-0.21,0.07,1.00,-0.82,-0.95,-0.66,-0.96,0.48,0.07,-0.98,0.09,-0.53,0.40,-0.38,0.68,-0.32,-0.43,-0.25,-0.02,-0.57,0.68,-1.00,0
3,0,0.00,0.00,0.00,0.00,0.00,0.00,0.61,-1.00,-0.74,-0.96,-0.89,-0.27,1.00,-0.48,-0.92,-0.70,-0.98,0.35,0.07,-0.99,-0.16,-0.53,-0.14,-0.69,0.95,-0.25,-0.43,-0.42,-0.02,-0.51,0.95,-1.00,0
4,1,0.78,0.00,0.00,1.00,0.00,1.00,0.00,-0.88,-0.68,-0.75,-0.81,-0.48,-0.79,-0.80,-0.89,-0.92,-0.44,-0.70,0.07,-0.98,0.90,-0.17,0.14,-0.34,0.79,-0.04,-0.43,0.32,-0.02,-0.33,0.79,-1.00,1


<a name="metrics"></a>
# Métrica de avaliação:
A métrica de avaliação utilizada para determinar o modelo de machine learning será o **ROC AUC**.<br>
* **ROC** do inglês *Receiver Operating Characteristics* é uma curva plotada entre os eixos *Taxa de Verdadeiros Positivos* e a *Taxa de Falsos Positivos*. Essa métrica mede o quanto o modelo consegue separar o binômio de saída entre suas duas classes (```ICU = 0``` e ```ICU = 1```).
* **AUC** do inglês *Area Under the Curve* é a maneira de resumir a curva **ROC** em um único valor real, de maneira a facilitar a comparação com outros modelos. Seria o mesmo que calcular a inegral do ROC<br>
* **ROC AUC**<br>
    * O valor será próximo de 0.5 se o modelo não tiver capacidade de separar as variáveis de saída em suas respectivas classes. Imagine que o modelo Tem 50% de verdadeiro positivo e 50% de falso positivo, ou seja, acerta em 50% das tentativas.
    * Quanto mais o valor se aproxima de 1.0, mais capacidade o modelo tem de separar os dois grupos, ou classes, acertando muito mais os Verdadeiros Positivos que errando.
    * O inverso também é verdadeiro. Quanto mais o valor se aproxima de 0, significa que o modelo também tem a capacidade de separar o binômio de saída em dois grupos, só que nesse caso, errando muito, pois teria um valor de Falsos Verdadeiros muito grande. Como a saída tem 2 estados, sendo 0 ou 1, poderia ser usado o inverso (NOT) da resposta e ainda assim o modelo acertaria bastante.

<a name="dummy"></a>
# Dummy Classifier
Essa etapa visa criar um modelo genérico que faz predições dos dados de entrada usando regras bem simples. Ter um modelo "bobo" que usa uma regra simples é importante para avaliar o modelo final, pois se espera do modelo a ser implementado que ao menos seja melhor que escolher aleatóriamente entre as alternativas de saída, ou então "chutar" a alternativa de saída mais comum.

In [5]:
run_model_cv('DummyClassifier', DummyClassifier, df, 10, 10)

DummyClassifier: AUC Mean: 0.5, AUC Std: 0.0, AUC CI: 0.5 - 0.5


Vemos que o DummyClassifier encontrou um ROC AUC Médio de **0.5**, logo, essa deve ser a meta inicial a ser batida.

<a name="ModSel"></a>
# Seleção de Modelo
A seleção de modelo será suportada pelo módulo LazyClassifier. A idéia aleatorizar os dados, dividi-los em teste e treino, e rodar a classe LazyClassifier, que vai treinar os modelos mais comuns e dar o resultado para cada um dos modelos.<br>
A função *many_Lazy_Classifiers* roda n vezes o teste e treino, trazendo como resultado a média e o desvio padrão de cada modelo, tornando assim facil a seleção de qual modelo utilizar.<br>
É importante salientar que o módulo LazyClassifier treina os dados utilizando valores padrões dos modelos, ou seja, sem 'tunar' os hiperparâmetros de cada modelo. Sabemos que esse tune pode melhorar muito o resultado de um modelo, fazendo com que algum que tivesse uma resposta não tão boa com os valores padrões, superasse o modelo com a melhor avaliação usando os parâmetros padrão.<br>
Para esse caso irá ser usado o modelo que der o melhor valor geral, e após isso, os hiperparâmetros desse modelo serão avaliados.


In [6]:
many_Lazy_Classifiers(df, 100)

100%|██████████████████████████████████████████████████████████████████████████████████| 44/44 [00:09<00:00,  4.81it/s]


,mean,std
Model,,
CatBoostClassifier,0.74,0.04
RandomForestClassifier,0.74,0.04
ExtraTreesClassifier,0.73,0.04
GradientBoostingClassifier,0.72,0.05
XGBClassifier,0.72,0.04
SVC,0.72,0.04
HistGradientBoostingClassifier,0.72,0.04
LGBMClassifier,0.72,0.04
NearestCentroid,0.72,0.04


Houve um impate em média e Desvio padrão entre dois modelos: **CatBoostClassifier** e **RandomForestClassifier**. Como o **RandomForestClassifier** é mais difundido e tem mais material sobre sua implementação, foi escolhido usar esse modelo.

<a name="rfcsf"></a>
# Random Forest Classifier


## imprimindo os parâmetros padrões
A idéia aqui não é fazer uma análise, e sim ter uma referência dos parâmetros *default* do modelo **RandonForestClassiier**.

In [29]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(random_state = 42)
from pprint import pprint
# Look at parameters used by our current forest
print('Os parâmetros sendo utilizados no modelo são::\n')
pprint(rf_clf.get_params())

Os parâmetros sendo utilizados no modelo são::

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


## Avaliação de performance com parâmetros padrão
A ideia é avaliar o desempenho do modelo **RandonForestClassifier** usando para isso os parâmetros padrão. Esse resultado vai servir de base para o *tunning* dos hiperparâmetros.<br>
Para fazer tal avaliação, pensando em minimizar o erro causado pelo baixo número de observações que se tem, será executada a função ```run_model_cv```, que usa ```RepeatedStratifiedKFold``` para estimar um intervalo de confiança do resultado.<br>

In [30]:
run_model_cv('RandomForestClassifier', rf_clf, df, 5, 30)

RandomForestClassifier: AUC Mean: 0.8, AUC Std: 0.053, AUC CI: 0.69 - 0.91


Podemos ver então que utilizando os valores padrão temos um **AUC Médio = 80%** e um **Desvio Padrão = 7.9%**.

## Seleção automática de hiperparâmetros
O método ```RandomizedSearchCV``` usa um grid de parâmetros pré configurado, e aleatóriamente utiliza esses parâmetros em seus modelos, encontrando assim a melhor combinação dos parâmetros sem necessáriamente rodar todas as combinações possíveis.<br>
A variável ```random_grid``` carrega esses parâmetros que se deseja testar.

In [20]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(2, 30, num = 5)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [1, 2, 5, 7, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
oob_score = [True, False]
criterion = ['gini', 'entropy']
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap,
               'oob_score' : oob_score,
               'criterion' : criterion}
pprint(random_grid)

{'bootstrap': [True, False],
 'criterion': ['gini', 'entropy'],
 'max_depth': [2, 9, 16, 23, 30, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 4, 7],
 'min_samples_split': [1, 2, 5, 7, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],
 'oob_score': [True, False]}


In [ ]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 500, cv = 5, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
rf_random.fit(df.drop('ICU', axis=1), df['ICU'])
rf_random.best_params_

Fitting 5 folds for each of 500 candidates, totalling 2500 fits


{'oob_score': False,
 'n_estimators': 800,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'criterion': 'gini',
 'bootstrap': False}

In [22]:
rf_clf = RandomForestClassifier(random_state = 42, n_estimators=1200, min_samples_split=7,
                                min_samples_leaf=1, max_features='log2', max_depth=None,
                                bootstrap=False, criterion='gini')
run_model_cv('RandomForestClassifier', rf_clf, df, 10, 50)
#RandomForestClassifier: AUC Mean: 0.8, AUC Std: 0.076, AUC CI: 0.65 - 0.95

In [27]:
rf_clf = RandomForestClassifier(random_state = 42, n_estimators=800, min_samples_split=10,
                                min_samples_leaf=1, max_features='sqrt', max_depth=None,
                                bootstrap=False, criterion='gini')
run_model_cv('RandomForestClassifier', rf_clf, df, 5, 30)

RandomForestClassifier: AUC Mean: 0.8, AUC Std: 0.052, AUC CI: 0.7 - 0.9


# Referências
* https://towardsdatascience.com/modeling-teaching-a-machine-learning-algorithm-to-deliver-business-value-ad0205ca4c86
* https://docs.microsoft.com/en-us/windows/ai/windows-ml/what-is-a-machine-learning-model
* https://medium.com/@datadrivenscience/7-stages-of-machine-learning-a-framework-33d39065e2c9
* https://developers.google.com/machine-learning/crash-course/classification/roc-and-auc
* https://towardsdatascience.com/understanding-auc-roc-curve-68b2303cc9c5
* https://medium.com/bio-data-blog/entenda-o-que-%C3%A9-auc-e-roc-nos-modelos-de-machine-learning-8191fb4df772
* https://pypi.org/project/lazypredict/
* https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74